In [4]:
#import libraries

import pandas as pd
import numpy as np
import seaborn as sb
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

import sklearn
from sklearn.model_selection import train_test_split
from pandas import Series, DataFrame
from pylab import rcParams


from sklearn.metrics import classification_report,accuracy_score
sb.set(style="white")
sb.set(style="whitegrid", color_codes=True)
import itertools
from sklearn import svm
import graphviz
from sklearn.model_selection import GridSearchCV,StratifiedKFold, LeaveOneOut
from sklearn.datasets import make_regression
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import warnings; warnings.simplefilter('ignore')

In [5]:
### reading data and printing the data ###
data = pd.read_csv('/home/sahulphaniraj/csce633-ml/MachineLearningProject/Dataset/classical_feature.csv',names=['ID','TruncatedID','Mean','SD','AUC','Peak','MaxIndex','Halfpk','Skew','HHMean','Fat','Carb','Protein'
])
print(data.shape)
print(list(data.columns))

(171, 13)
['ID', 'TruncatedID', 'Mean', 'SD', 'AUC', 'Peak', 'MaxIndex', 'Halfpk', 'Skew', 'HHMean', 'Fat', 'Carb', 'Protein']


In [6]:
### CHO
CHO_data = data.drop(['ID','Protein','Fat'], axis=1) #taking only the Carb target column
print(CHO_data.Carb.unique())# Finding the unique values in that column
print(CHO_data.TruncatedID.unique()) #finiding unique IDs

[51 48 54]
['2133-002' '2133-022' '2133-020' '2133-004' '2133-010' '2133-015'
 '2133-040' '2133-013' '2133-008' '2133-019' '2133-001' '2133-024'
 '2133-011' '2133-018' '2133-006' '2133-032' '2133-025' '2133-017'
 '2133-028' '2133-039' '2133-021' '2133-030' '2133-033' '2133-037'
 '2133-026' '2133-012' '2133-036' '2133-035' '2133-041' '2133-009']


In [10]:
###for LOW-HIGH

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([48,54])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Carb == 48, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Carb== 54, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loss

final_accuracy =[] #will be used for finding mean across nested cross validation
F1score = []

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #will print the current fold number, this is for initial check, can delete it
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    for C_val in range(1,20): #ranges of decision tree depth
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = svm.SVC(C=C_val,kernel='linear') #SVM classifier
            #convert dataframe to array
            #xtr_arr = xtr.as_matrix(columns=None)
            #ytr_arr = ytr.as_matrix(columns=None)
            #clf = clf.fit(xtr_arr,ytr_arr)
            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    #best SVM classifier
    print(best_param)
    clf = svm.SVC(C=best_param,kernel='linear') 
    clf=clf.fit(X_train,Y_train)
    Y_pred = clf.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
    dic = classification_report(Y_test,Y_pred,output_dict=True)
    dic2 = dic['macro avg']
    F1score.append(dic2['f1-score'])
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)
print("The average F1-score is : ", np.mean(F1score))#final result (mean of accuracies in outer CV)


For fold : 1
1
For fold : 2
1
For fold : 3
1
For fold : 4
7
For fold : 5
6
For fold : 6
14
For fold : 7
1
For fold : 8
2
For fold : 9
1
For fold : 10
3
For fold : 11
2
For fold : 12
1
For fold : 13
1
For fold : 14
3
For fold : 15
5
For fold : 16
1
For fold : 17
4
For fold : 18
1
For fold : 19
10
For fold : 20
1
For fold : 21
13
For fold : 22
1
For fold : 23
1
For fold : 24
2
For fold : 25
15
For fold : 26
18
For fold : 27
1
For fold : 28
5
For fold : 29
2
For fold : 30
1
The final accuracy is :  0.6666666666666666
The average F1-score is :  0.6666666666666666


In [11]:
#MEDIUM-HIGH

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([51,54])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Carb == 51, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Carb== 54, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loss

final_accuracy =[] #will be used for finding mean across nested cross validation
F1score = []

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #will print the current fold number, this is for initial check, can delete it
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    for C_val in range(1,20): #ranges of decision tree depth
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = svm.SVC(C=C_val,kernel = 'linear') #SVM classifier
            #convert dataframe to array
            #xtr_arr = xtr.as_matrix(columns=None)
            #ytr_arr = ytr.as_matrix(columns=None)
            #clf = clf.fit(xtr_arr,ytr_arr)
            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    #best SVM classifier
    print(best_param)
    clf = svm.SVC(C=best_param, kernel='linear') 
    clf=clf.fit(X_train,Y_train)
    Y_pred = clf.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
    dic = classification_report(Y_test,Y_pred,output_dict=True)
    dic2 = dic['micro avg']
    F1score.append(dic2['f1-score'])
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)
print("The average F1-score is : ", np.mean(F1score))#final result (mean of accuracies in outer CV)



For fold : 1
12
For fold : 2
3
For fold : 3
12
For fold : 4
18
For fold : 5
12
For fold : 6
3
For fold : 7
5
For fold : 8
4
For fold : 9
12
For fold : 10
10
For fold : 11
5
For fold : 12
3
For fold : 13
5
For fold : 14
8
For fold : 15
5
For fold : 16
5
For fold : 17
10
For fold : 18
3
For fold : 19
16
For fold : 20
12
For fold : 21
14
For fold : 22
7
For fold : 23
6
For fold : 24
11
For fold : 25
5
For fold : 26
2
For fold : 27
9
For fold : 28
6
For fold : 29
7
For fold : 30
4
The final accuracy is :  0.775
The average F1-score is :  0.775


In [14]:
#LOW-MEDIUM

CHO_LH_data = CHO_data[CHO_data['Carb'].isin([51,48])] #considering the two classes for binary classification
CHO_LH_data.loc[CHO_LH_data.Carb == 51, 'CHO_bin'] = 0 #giving them binary values, as double variables sometimes create error
CHO_LH_data.loc[CHO_LH_data.Carb== 48, 'CHO_bin'] = 1
CHO_LH_data = CHO_LH_data.drop(['Carb'], axis=1)
#the above 4 lines will be different for different classification, the remaining code will be same
X = CHO_LH_data.loc[:, CHO_LH_data.columns != 'CHO_bin'] #Feature set
Y = CHO_LH_data.loc[:, CHO_LH_data.columns == 'CHO_bin'] #target value/label

values =['2133-002', '2133-022', '2133-020', '2133-004', '2133-010', '2133-015',
 '2133-040' ,'2133-013' ,'2133-008' ,'2133-019' ,'2133-001', '2133-024',
 '2133-011' ,'2133-018', '2133-006', '2133-032' ,'2133-025', '2133-017',
 '2133-028' ,'2133-039', '2133-021', '2133-030', '2133-033', '2133-037',
 '2133-026', '2133-012' ,'2133-036', '2133-035', '2133-041' ,'2133-009'] #needed for manual loss

final_accuracy =[] #will be used for finding mean across nested cross validation
F1score = []

for i in range(0,30): #as we have 30 IDs
    print("For fold :", i+1) #will print the current fold number, this is for initial check, can delete it
    test =CHO_LH_data[CHO_LH_data['TruncatedID'].isin([values[i]])] #test data    
    train =CHO_LH_data[~CHO_LH_data['TruncatedID'].isin([values[i]])]#train data
    X_train = train.loc[:, train.columns != 'CHO_bin'] #test feature
    Y_train = train.loc[:, train.columns == 'CHO_bin'] #test label
    X_test = test.loc[:, test.columns != 'CHO_bin']#train feature    
    Y_test =test.loc[:, test.columns == 'CHO_bin']#train label
    
    train_ids = X_train['TruncatedID']#needed for inner cross-validation
    train_ids = np.array(train_ids)#needed for inner cross-validation
    train_ids = np.unique(train_ids)#needed for inner cross-validation
    X_train = X_train.drop(['TruncatedID'],axis = 1) #dropping this column as it is not a feature for classification
    X_test = X_test.drop(['TruncatedID'],axis = 1)#dropping this column as it is not a feature for classification
    max_acc = 0 #initializing maximum accuracy which will be needed later    
    ypred_all = []
    for C_val in range(1,20): #ranges of decision tree depth
        ypred_all = []
        for j in range(0,29): #as inner cross validation ha 29 IDs
            inner_test = train[train['TruncatedID'].isin([train_ids[j]])] #validation set
            inner_train = train[~train['TruncatedID'].isin([train_ids[j]])]#inner train set
            xtr = inner_train.loc[:, inner_train.columns != 'CHO_bin'] #inner train set features
            ytr = inner_train.loc[:, inner_train.columns == 'CHO_bin']# inner train set label
            xvl = inner_test.loc[:, inner_test.columns != 'CHO_bin']#validation set features
            yvl = inner_test.loc[:, inner_test.columns == 'CHO_bin']#validation set labels
            xtr = xtr.drop(['TruncatedID'],axis = 1)#We dont need this column for classification        
            xvl = xvl.drop(['TruncatedID'],axis = 1)#We dont need this column for classification
            clf = svm.SVC(C=C_val,kernel='linear') #SVM classifier
            #convert dataframe to array
            #xtr_arr = xtr.as_matrix(columns=None)
            #ytr_arr = ytr.as_matrix(columns=None)
            #clf = clf.fit(xtr_arr,ytr_arr)
            clf=clf.fit(xtr,ytr)
            ypred = clf.predict(xvl)#prediction
            ypred_all.append(accuracy_score(yvl,ypred))
        ypred_mean = np.mean(ypred_all)#finding mean of all accuracy
        if max_acc<ypred_mean:#finding the depth which gives maximum accuracy
            max_acc = ypred_mean
            best_param = C_val
    #best SVM classifier
    print(best_param)
    clf = svm.SVC(C=best_param,kernel='linear') 
    clf=clf.fit(X_train,Y_train)
    Y_pred = clf.predict(X_test)
    final_accuracy.append(accuracy_score(Y_test,Y_pred))
    dic = classification_report(Y_test,Y_pred,output_dict=True)
    print(dic)
    dic2 = dic['micro avg']
    F1score.append(dic2['f1-score'])
print("The final accuracy is : ", np.mean(final_accuracy))#final result (mean of accuracies in outer CV)
print("The average F1-score is : ", np.mean(F1score))#final result (mean of accuracies in outer CV)

For fold : 1
1
{'0.0': {'precision': 0.6666666666666666, 'recall': 1.0, 'f1-score': 0.8, 'support': 2}, '1.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'micro avg': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 3}, 'macro avg': {'precision': 0.3333333333333333, 'recall': 0.5, 'f1-score': 0.4, 'support': 3}, 'weighted avg': {'precision': 0.4444444444444444, 'recall': 0.6666666666666666, 'f1-score': 0.5333333333333333, 'support': 3}}
For fold : 2
14
{'0.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, '1.0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 2}, 'micro avg': {'precision': 0.5, 'recall': 0.5, 'f1-score': 0.5, 'support': 4}, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 4}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 4}}
For fold : 3
1
{'0.0': {'pre

1
{'0.0': {'precision': 0.6666666666666666, 'recall': 1.0, 'f1-score': 0.8, 'support': 2}, '1.0': {'precision': 1.0, 'recall': 0.5, 'f1-score': 0.6666666666666666, 'support': 2}, 'micro avg': {'precision': 0.75, 'recall': 0.75, 'f1-score': 0.75, 'support': 4}, 'macro avg': {'precision': 0.8333333333333333, 'recall': 0.75, 'f1-score': 0.7333333333333334, 'support': 4}, 'weighted avg': {'precision': 0.8333333333333333, 'recall': 0.75, 'f1-score': 0.7333333333333334, 'support': 4}}
For fold : 20
13
{'0.0': {'precision': 0.5, 'recall': 0.5, 'f1-score': 0.5, 'support': 2}, '1.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'micro avg': {'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1-score': 0.3333333333333333, 'support': 3}, 'macro avg': {'precision': 0.25, 'recall': 0.25, 'f1-score': 0.25, 'support': 3}, 'weighted avg': {'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1-score': 0.3333333333333333, 'support': 3}}
For fold : 21
1
{'0